In [ ]:
!export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda
!export CUDA_DIR="/usr/lib/cuda"
!export TF_GPU_ALLOCATOR=cuda_malloc_async

In [ ]:
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(0)
tf.config.threading.set_intra_op_parallelism_threads(0)
physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8000)])
# tf.config.experimental.set_virtual_device_configuration(physical_devices[1], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)
# print("GPUS: {}".format(len(physical_devices)))

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow import keras
import time
import tensorflow_addons as tfa

layers = tf.keras.layers

from IPython import display

In [ ]:
# (train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
# train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
# train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
# !rm -rf ./logs

In [ ]:
def normalize(image):
    return (image -127.5) / 127.5

def denormalize(image):
    return tf.cast(image * 127.5 + 127.5, np.uint8)

In [ ]:
import pathlib
import tensorflow as tf
import random

In [ ]:
input_w = 720
input_h = 480


# input_h = 672
# input_w = 976

# input_h *= 1.2
# input_w *= 1.2


# input_h = 576
# input_w = 864

hwfactor = input_h / input_w

input_h, input_w

BATCH_SIZE = 5

EPOCHS=400
STEPS_PER_EPOCH = 500

In [ ]:
input_h = int(input_h)
input_w = int(input_w)
input_h, input_w

# Enlarge image

In [ ]:
rf = 1
resize_factor = 1/rf # about 0.0833
resized_size_h = int(input_h * resize_factor)
resized_size_w = int(input_w * resize_factor)

print(resized_size_h, resized_size_h * rf, input_h)
assert resized_size_h * rf == input_h

print(resized_size_w, resized_size_w * rf, input_w)
assert resized_size_w * rf == input_w

## Generate and save random images

In [ ]:
import os
import re

def image_generator(pictures = [], large_pictures = [], ds_pictures = [], return_ext=False):
    def parse_name(filename):
        return "{}_" + re.sub(r"\.(png|jpeg|jpg)", r"_{}.\1", os.path.basename(filename))
    
    def format_name(name, subname):
        return name.format(random.randint(0, 1000), subname)
    def fn():
        for filename in pictures:
            fname = parse_name(filename)
            raw_png = tf.io.read_file(str(filename), name=filename)
            decoded_png_2 = tf.image.decode_png(raw_png, channels=3, name=filename)
            decoded_png_2 = tf.image.resize(decoded_png_2, [input_h, input_w],
                              method=tf.image.ResizeMethod.BILINEAR)

            yield decoded_png_2, format_name(fname, "original")
            yield tf.image.flip_left_right(decoded_png_2), format_name(fname, "flipped")

        for filename in ds_pictures:
            fname =  parse_name(filename)
            raw_png = tf.io.read_file(str(filename), name=filename)
            decoded_png = tf.image.decode_jpeg(raw_png, channels=3, name=filename)
            cropped = tf.image.resize_with_crop_or_pad(
              decoded_png,input_h ,input_w
            )
            yield cropped, format_name(fname, "original")
            yield tf.image.flip_left_right(cropped), format_name(fname, "center_crop")

            for i in range(200):
                cropped = tf.image.random_crop(
                  decoded_png, size=[input_h, input_w, 3])
                yield cropped, format_name(fname, "random_crop_"+str(i))
                yield tf.image.flip_left_right(cropped), format_name(fname, "random_crop_and_flip_"+str(i))

        for filename in large_pictures:
            fname =  parse_name(filename)
            raw_png = tf.io.read_file(str(filename), name=filename)
            decoded_png = tf.image.decode_jpeg(raw_png, channels=3, name=filename)
            cropped = tf.image.resize_with_crop_or_pad(
              decoded_png,input_h ,input_w
            )
            yield cropped, format_name(fname, "center_crop")

            for i in range(20):
                cropped = tf.image.random_crop(
                  decoded_png, size=[input_h, input_w, 3])
                yield cropped, format_name(fname, "random_crop_"+str(i))
            # yield tf.image.random_flip_left_right(cropped)
    return fn

In [ ]:
from IPython.display import clear_output
from multiprocessing import Lock, Process, Queue, current_process, Value, Pool, cpu_count
from concurrent import futures

def cache_images():
    data_dir = pathlib.Path("./prepa")
    pictures = list(data_dir.glob('*.png'))


    data_dir = pathlib.Path("./people")
    large_pictures = list(data_dir.glob('*.jpg'))


    data_dir = pathlib.Path("./ds_images")
    ds_pictures = list(data_dir.glob('*.jpg'))
    
    approx_size = len(pictures)*2 + len(ds_pictures)*2 + len(ds_pictures)*200*2 + len(large_pictures) + len(large_pictures)*20
    
    gen = image_generator(pictures, large_pictures, ds_pictures)
    
    
    print("starting")
    counter = Value('i', 0)
    
        
    it = iter(gen())
    con = True
    
    np = cpu_count()
    print(f'You have {np} cores')



    def f1(im, filename):
            im.numpy()
            tf.keras.utils.save_img('./tmp/{}'.format(filename), im)
            with counter.get_lock():
                counter.value += 1

            if (counter.value % 10 == 0):
                clear_output(wait=True)
                print(f"{counter.value+1}/{approx_size}")   
            return f"{im.shape} {filename}"
    with futures.ThreadPoolExecutor(max_workers=16) as executor:    
        for im, filename  in it:
#             im, filename = next(it)
            future = executor.submit(f1, im, filename)
#             future.add_done_callback(lambda x: print(f"donee: {x}"))
        try:
            data = future.result()
        except Exception as exc:
            print('generated an exception: %s' % exc)
        else:
            print('%r page is' % data)
    

# cache_images()

## Load the cached images

In [ ]:
import random

def image_generator_cached(size=None):
    data_dir = pathlib.Path("./tmp")
    all_files = []
    for filename in list(data_dir.glob('*.png')):
        all_files.append((filename, 'png'))
    for filename in list(data_dir.glob('*.jpg')):
        all_files.append((filename, 'jpg'))
    
    random.shuffle(all_files)
    if size:
        all_files = all_files[:size]
    
    def fn():
        for filename, img_type in all_files:
            raw_image = tf.io.read_file(str(filename), name=filename)
            if img_type == "png":
                yield tf.image.decode_png(raw_image, channels=3, name=filename)
            elif img_type == "jpg":
                yield tf.image.decode_jpeg(raw_image, channels=3, name=filename)
            
    return fn

In [ ]:
image_count = len(list(pathlib.Path("./tmp").glob('*')))
print(f"image_count={image_count}")

fraction = int((STEPS_PER_EPOCH * EPOCHS) / image_count)
print(fraction)

dataset_size = int((STEPS_PER_EPOCH * EPOCHS) / (fraction + 1))
print(f"better dataset size: {dataset_size}")


In [ ]:

def make_dataset(size=None):
    load_large_images = image_generator_cached(size)

    train_ds = tf.data.Dataset.from_generator(load_large_images,  output_signature=
         tf.TensorSpec(shape=(input_h, input_w, 3), dtype=tf.float16)).map(normalize)

    def resize_and_couple(images):
        # return (images,images)
        down = tf.image.resize(
            images,
            [int(resized_size_h / 8), int(resized_size_w / 8)],
            preserve_aspect_ratio=True,
            antialias=False,
            name=None)


        return (images, tf.image.resize(
            down,
            [resized_size_h, resized_size_w],
            preserve_aspect_ratio=True,
            antialias=False,
            name=None))

    zipped_train_dataset = train_ds.interleave(
      lambda x: tf.data.Dataset.from_tensors(x).map(resize_and_couple, num_parallel_calls=tf.data.AUTOTUNE),
      cycle_length=4, num_parallel_calls=tf.data.AUTOTUNE,
      deterministic=False
    )
    # Batch and shuffle the data
    return zipped_train_dataset
    # train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(1)

# lp_imgs = list(train_ds.shuffle(200).take(2))
train_ds = make_dataset(dataset_size).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
def get_random_samples(size=2):
    return make_dataset(size)

In [ ]:
resized_size_h

In [ ]:
model_input_w = input_w
model_input_h = input_h

model_input_w, model_input_h

In [ ]:
resized_size_w, resized_size_h

In [ ]:
input_w, input_h

In [ ]:
def closest(stride, base_kernel, K):
    lst = list(range(0,base_kernel+stride, stride))
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

closest(2, 6, 5)

In [ ]:
import math

def get_grid_size(x):
    col = int(math.sqrt(x))
    row = int(x / col)

    y = int(x - (col*row))
    row+=y

    return row, col

def get_bi_column(x, col=2):
    row = int(x / col)

    y = int(x - (col*row))
    row+=y

    return row, col

get_bi_column(9)

In [ ]:
import random

num_random_samples = 10
random_samples_ds = get_random_samples(num_random_samples)

In [ ]:
initializer = tf.keras.initializers.LecunNormal()
initializer(shape=(2, 2))

In [ ]:
def conv(filters=3, kernel_size=(1,1), name=None, kernel_initializer="Zeros", normalize=False, activation=None, strides=(1,1)):
    act = {
        "leaky_relu": tf.nn.leaky_relu,
        "tanh": tf.nn.tanh
        }
    def fn(x):
        x = layers.Conv2D(3, kernel_size=kernel_size, kernel_initializer=kernel_initializer, strides=strides, padding="same", name=name)(x)
        if (normalize):
            x = layers.BatchNormalization()(x)
        if activation and activation in act:
            print(activation)
            x = act[activation](x)
        return x
    return fn

In [ ]:
def test_cnvs():
    base_f = 1
    kernel_size = closest(base_f, base_f, base_f*hwfactor)

    kernel_size_100 = closest(100, 100, 100*hwfactor)

    convs = [
        # ("Zeros (kernel=100)", conv(kernel_size=kernel_size_100)),
        # ("Zeros", conv(kernel_size=kernel_size)),
        # ("Zeros (norm)", conv(kernel_size=kernel_size, normalize=True)),
        # ("Zeros (norm & relu)", conv(kernel_size=kernel_size, normalize=True, activation="leaky_relu")),
        # ("Zeros (norm & relu, stride=10)", conv(kernel_size=kernel_size, normalize=True, strides=(10,10), activation="leaky_relu")),
        ("LecunNormal", conv(kernel_initializer=tf.keras.initializers.LecunNormal(seed=123))),
        ("LecunNormal (kernel=3)", conv(kernel_size=closest(3, 3, 3*hwfactor), kernel_initializer=tf.keras.initializers.LecunNormal(seed=123))),
        ("LecunNormal (norm)", conv(kernel_initializer=tf.keras.initializers.LecunNormal(seed=123), normalize=True)),
        ("LecunNormal (norm & relu)", conv(kernel_initializer=tf.keras.initializers.LecunNormal(seed=123), normalize=True, activation="leaky_relu")),
        ("LecunNormal (kernel=3, norm & relu)", conv(kernel_size=closest(3, 3, 3*hwfactor), kernel_initializer=tf.keras.initializers.LecunNormal(seed=123), normalize=True, activation="leaky_relu")),
        # ("VarianceScaling", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.VarianceScaling())),
        # ("VarianceScaling (norm)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.VarianceScaling(), normalize=True)),
        # ("VarianceScaling (norm & relu)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.VarianceScaling(), normalize=True, activation="leaky_relu")),
        # ("LecunUniform", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.LecunUniform())),
        # ("LecunUniform (norm)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.LecunUniform(), normalize=True)),
        # ("LecunUniform (norm & relu)", conv(kernel_size=kernel_size, kernel_initializer=tf.keras.initializers.LecunUniform(), normalize=True, activation="leaky_relu"))
    ]


    for images, resized_images in random_samples_ds.shuffle(125).take(2):
        plt.figure()
        # plt.figure(figsize=(12,4))
       
        # im[:,:,1] = 1
        # im[:,:,2] = 1
        # plt.imshow(im)
        # print(images.shape)
        # down_model = downsample(3, 4)
        # down_result = down_model(tf.cast(tf.expand_dims(images, axis=0), np.float32))
        # print (down_result.shape)

        # plt.figure()
        # plt.title("down_result")
        # plt.imshow(denormalize(np.squeeze(down_result)))

        # up_model = upsample(3, 4)
        # up_result = up_model(down_result)
        # print (up_result.shape)

        # plt.figure()
        # plt.title("up_result")
        # plt.imshow(denormalize(np.squeeze(up_result)))
        
        
        for (cname, cfn) in convs:
            im = np.copy(images)
            conv_image = tf.squeeze(cfn(tf.cast(tf.expand_dims(im, axis=0), np.float16)))
            pp = lambda x: (np.max(np.unique(x)), np.min(np.unique(x)))
            print(cname, pp(im), pp(conv_image))
            plt.figure(figsize=(12,4))
            plt.subplot(1,3,1)
            plt.title("Original")
            plt.imshow(denormalize(im))
            plt.subplot(1,3,2)
            plt.title(f"Conv: {cname}")
            plt.imshow(denormalize(conv_image))
            plt.axis('off')
            plt.subplot(1,3,3)
            plt.title(f"Original + Conv: {cname}")
            if conv_image.shape != im.shape:
                im = tf.image.resize(im, [conv_image.shape[0], conv_image.shape[1]])
            plt.imshow(denormalize(tf.add(conv_image, im)))
            plt.axis('off')

# test_cnvs()

In [ ]:


def test_generator(generator, discriminator, size=2):
    # test_dataset = tf.data.Dataset.zip((train_ds, resized_ds))
    

    sample = random.randrange(size, num_random_samples-1)
    test_data = random_samples_ds.shuffle(sample).take(size).batch(size)
    for images, resized_images in test_data:
        generated_images = generator(resized_images, training=False)

        print(f"images={images.shape}, resized_images={resized_images.shape}, generated_images={generated_images.shape}")

        imgs = zip(images, resized_images, generated_images)

        col, row = get_bi_column(size*3, 3)

        # print(f"col={col}, row={row}")

        fig = plt.figure(figsize=(row * 10, col * 7))

        i = 0
        for img_set in imgs:
            for img in img_set:
                plt.subplot(col, row, i+1)
                im = np.copy(img)
                im = denormalize(im)
                plt.imshow(im)
                plt.axis('off')
                i += 1

        decision = discriminator(generated_images)
        print(f"Decision shape: {decision.shape}")
        print (f"Decision for the scaled images: {decision}")


    return generated_images

In [ ]:
def test_architecture(down_stack, up_stack, v=False):
    sample = random.randrange(0, image_count-1)
    test_data = random_samples_ds.shuffle(1).take(1).batch(1)
    def p(x):
        if v:
            print(x)
            
    for images, inputs in test_data:
           
#         inputs = tf.constant(range(input_w*input_h*2*3), tf.float16)
#         inputs = tf.reshape(inputs, (2,input_w, input_h, 3))

    #     print((2,input_w, input_h, 3))
        p(inputs.shape)

        x = inputs

        skips = []
        i = 0
        for down in down_stack:
            p(f"Down: {i}")
            x = down(x)
            i+=1
            p(x.shape)
            skips.append(x)

        skips = reversed(skips[:-1])

        i = 0
        for up, skip in zip(up_stack, skips):
            p(f"Up: {i}")
            x = up(x)
            p(x.shape)
            i+=1

        assert upsample(3, 3)(x).shape == inputs.shape
    #     x = tf.keras.layers.Concatenate()([x, skip])

In [ ]:
def make_kernel(s):
        return closest(min(2,s), s, s*hwfactor), s

def pixel_shuffle(scale):
    return lambda x: tf.nn.depth_to_space(x, scale)

init_fn = tf.keras.initializers.LecunUniform(seed=123)

def residual_block(block_input, filters=64, momentum=0.8):
    x = layers.Conv2D(filters, kernel_size=3, padding='same', kernel_initializer=init_fn)(block_input)
    x = layers.BatchNormalization(momentum=momentum)(x)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    x = layers.Conv2D(filters, kernel_size=3, padding='same', kernel_initializer=init_fn)(x)
    x = layers.BatchNormalization(momentum=momentum)(x)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    x = layers.Add()([block_input,x]) 
    return x

def pixel_shuffle(x, channels, downsampleFactor):
    if downsampleFactor == 1:
        x = layers.Conv2D(channels * (downsampleFactor ** 2), 3, padding="same",
               activation="relu", kernel_initializer="Orthogonal",
                         strides=2)(x)
        outputs = tf.nn.depth_to_space(x, 2)
    else:
        x = layers.Conv2D(channels * (downsampleFactor ** 2), 3, padding="same",
               activation="relu", kernel_initializer="Orthogonal")(x)
        outputs = tf.nn.depth_to_space(x, downsampleFactor)
    
    return outputs

def upscale_block(block_input, filters=256, scale=1):
# def upscale_block(block_input, filters=128, scale=1):
    x = layers.Conv2D(filters, kernel_size=3, padding='same', kernel_initializer=init_fn)(block_input)
    x = pixel_shuffle(x, filters, 1)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    return x



def make_sgenerator_model(scale=8, num_filters=64):
    # needs to be divisible by the stride to avoid checkerboard patterns
    block_size = 64
    

    inputs = tf.keras.Input(shape=(None, None, 3))
    pre_x = layers.Conv2D(block_size, kernel_size=9, padding='same', kernel_initializer=init_fn)(inputs)
    pre_x = layers.PReLU(shared_axes=[1, 2])(pre_x)
    
    x = residual_block(pre_x)
    x = residual_block(x)
    x = residual_block(x)
    x = residual_block(x)
    x = residual_block(x)
    
    x = layers.Conv2D(block_size, kernel_size=3, padding='same', kernel_initializer=init_fn)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([pre_x,x])
    
    x = upscale_block(x)
    x = upscale_block(x)
    
    x = layers.Conv2D(3, kernel_size=9, padding='same', kernel_initializer=init_fn)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.PReLU(shared_axes=[1, 2])(x)
    
    return tf.keras.Model(inputs, x, name="generator")
# test_model = make_sgenerator_model()

In [ ]:
x = (input_w, input_h)
x = x[0] / 2, x[1] / 2
print(x)
x = x[0] / 2, x[1] / 2
print(x)
x = x[0] / 2, x[1] / 2
print(x)
x = x[0] / 2, x[1] / 2
print(x)
x = x[0] / 3, x[1] / 2
print(x)
x = x[0] / 3, x[1] / 3
print(x)

In [ ]:
# def make_discriminator_model():
#     model = tf.keras.Sequential()
#     model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
#                                      input_shape=[input_h, input_w, 3]))
#     model.add(layers.LeakyReLU())
#     model.add(layers.Dropout(0.3))

#     model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
#     model.add(layers.LeakyReLU())
#     model.add(layers.Dropout(0.3))

#     model.add(layers.Flatten())
#     model.add(layers.Dense(1))

#     return model

def disc_bloc(filters, strides):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same', kernel_initializer=init_fn))
    model.add(layers.LeakyReLU())
    return model
    

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, padding='same', kernel_initializer=init_fn))
    model.add(layers.LeakyReLU())
    
    model.add(disc_bloc(64, 2))
    model.add(disc_bloc(128, 1))
    model.add(disc_bloc(128, 2))
    model.add(disc_bloc(256, 1))
    model.add(disc_bloc(256, 2))
    model.add(disc_bloc(512, 1))
    model.add(disc_bloc(512, 2))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU())
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    return model


In [ ]:
discriminator = make_discriminator_model()
generator = make_sgenerator_model()

discriminator.build(input_shape=[2, input_h, input_w, 3])
print(discriminator.output_shape)

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [ ]:
cross_entropy(tf.constant([255,255], tf.float16),tf.constant([1,1], tf.float16))

In [ ]:
# def bce_loss(self, real, pred):
#     # compute binary cross entropy loss without reduction
#     BCE = tf.keras.losses.BinaryCrossentropy(reduction=Reduction.NONE)
#     loss = BCE(real, pred)
#     # compute reduced mean over the entire batch
#     loss = reduce_mean(loss) * (1. / self.numReplicas)
#     # return reduced bce loss
#     return loss

# def mse_loss(self, real, pred):
#     # compute mean squared error loss without reduction
#     MSE = tf.keras.losses.MeanSquaredError(reduction=Reduction.NONE)
#     loss = MSE(real, pred)
#     # compute reduced mean over the entire batch
#     loss = reduce_mean(loss) * (1. / self.numReplicas)
#     # return reduced mse loss
#     return loss

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [ ]:
# Optimizers

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])


# Metrics

In [ ]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'logs/gan/train/{current_time}'
train_log_dir

In [ ]:
def generate_and_save_images(generator, discriminator, epoch, save=True): 
  predictions = test_generator(generator, discriminator, 3)
  if save == True:
    plt.savefig('./gan_output/image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()


generate_and_save_images(generator, discriminator, 0, False)

In [ ]:
import signal
import sys

def sigint_handler(signal, frame):
    print ('KeyboardInterrupt is caught')
    checkpoint.save(file_prefix = checkpoint_prefix)
    sys.exit(0)
signal.signal(signal.SIGINT, sigint_handler)

In [ ]:
# lrgen=tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=[1000, 10000], values=[0.1, 1e-4, 1e-5])
# lrdist=tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=[1000, 10000], values=[1e-3, 1e-4, 1e-5])

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)


In [ ]:
checkpoint_dir = './gan4_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=5)

In [ ]:
try:
    if ckpt_manager.latest_checkpoint:
        checkpoint.restore(ckpt_manager.latest_checkpoint)
except:
    print("Could not restore the checkopint")

In [ ]:
# gpus = tf.config.list_logical_devices('GPU')

# with tf.device(gpus[0].name):

class GANModel(tf.keras.Model):
  def __init__(self, gen, disc):
    super(GANModel, self).__init__(name="GANModel")
    self.generator = gen
    self.discriminator = disc

  def train_step(self, all_images):
    images, resized = all_images
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(resized, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        
        disc_loss = discriminator_loss(real_output, fake_output)

        _gen_loss = gen_loss
        
        images = tf.cast(images, tf.float32)
        
        #         # avoid too large errors
        real_output = tf.clip_by_value(real_output, 0.0, 255.0)
        fake_output = tf.clip_by_value(fake_output, 0.0, 255.0)
        
        dimages = denormalize(images)
        dgenerated_images = denormalize(generated_images)
        ssim = tf.math.reduce_sum(tf.image.ssim(dimages, dgenerated_images, 255.0))
        ms_ssim = tf.math.reduce_sum(tf.image.ssim_multiscale(dimages, dgenerated_images, 255.0))
        same_ms_ssim = tf.math.reduce_sum(tf.image.ssim_multiscale(dimages, dimages, 255.0))
        
        ssim_loss = (same_ms_ssim - ssim)
        ms_ssim_loss = (same_ms_ssim - ms_ssim)
        ssim_losses =  tf.math.minimum(gen_loss, ssim_loss + ms_ssim_loss)
        
#         gen_loss += ssim_losses
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return {"gen_loss": gen_loss, "disc_loss":disc_loss, "_gen_loss": _gen_loss, "ssim_loss":ssim_loss,
            "ms_ssim_loss":ms_ssim_loss, "ssim":ssim, "ms_ssim":ms_ssim, "same_ms_ssim":same_ms_ssim }


In [ ]:
# %tensorboard --logdir logs/gradient_tape

In [ ]:
def on_epoch_end(epoch, logs=None):
    clear_output(wait=True)
    generate_and_save_images(generator, discriminator,
                    epoch + 1)
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                ckpt_save_path))
                                                        
lm = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)


tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = train_log_dir,
      write_graph=True, # visualize the graph
     histogram_freq = 1, update_freq=100,
#      profile_batch = (1,200)
                                                )

# train(train_ds, EPOCHS)
model = GANModel(generator, discriminator)
model.compile(metrics=["gen_loss", "disc_loss", "ms_ssim", "ms"])
# model.compile(metrics=["gen_loss", "disc_loss", "ms_ssim", "ms"], run_eagerly=True)
model.fit(train_ds.repeat(), epochs=EPOCHS, callbacks=[tboard_callback, lm], steps_per_epoch=STEPS_PER_EPOCH, workers=3)


In [ ]:
## def change_model(model, new_input_shape=(None, 40, 40, 3)):
    # replace input shape of first layer
    # model.layers[1].batch_input_shape = new_input_shape
    # input_layer = layers.InputLayer(input_shape=new_input_shape, name="input_1")
    # model.input = input_layer

    new_model = make_sgenerator_model(new_input_shape)

    # feel free to modify additional parameters of other layers, for example...
    # model._layers[2].pool_size = (8, 8)
    # model._layers[2].strides = (8, 8)

    # rebuild model architecture by exporting and importing via json
    # new_model = keras.models.model_from_json(model.to_json())
    new_model.summary()

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    # test new model on a random input image

    return new_model

In [ ]:
checkpoint_path = checkpoint_dir+"/weights/weights-{epoch:04d}.ckpt"

generator.save_weights(checkpoint_path.format(epoch=0))
discriminator.save_weights(checkpoint_path.format(epoch=0))

In [ ]:
save_path = "./weights/gan/2/{}/"

gen_path = os.path.join(save_path.format("generator"))
tf.saved_model.save(generator, gen_path)
disc_path = os.path.join(save_path.format("discriminator"))
tf.saved_model.save(discriminator, disc_path)


In [ ]:
loaded_generator = tf.saved_model.load(gen_path)
loaded_discriminator = tf.saved_model.load(disc_path)

In [ ]:
generate_and_save_images(loaded_generator, loaded_discriminator,
                             9999)